## TO-DO/BUGS/Considerations

### BUGS

- When introducing the ECG as feature, the prediction gives nan values. Need to try to implement a single feature LSTM autoencoder on the ECG and see if the behaviour is different

- Pad sequences with the mean of the feature may force the autoencoder to learn the mean instead the original ditribution. This problem occurs with HR,RR and DBP. TIME_STEP has to be setted to minimum lenght of features sequences.


### TO-DO

- Describe methodology: holdout method, preprocessing, multivariate lstm autoencoder, loss function (mse), reconstruction error, time for training, mean reconstructino error approach and adaptive thresholding based on it. Experimental results.

- 'if item > 0' condition when flattening inside normalization functions should be in clean_data function


### Questions

- Should I use also the std of reconstruction error for threshold estimation, as described in the work Adaptive Threshold for Outlier Detection on Data Streams?

### Considerations

- Nel caricare i dati la maggior parte dei sample era costituito da valori NaN. Una volta rimossi si ottengono dei valori con dei picchi, dovuti all'aver reso contigui valori che prima non lo erano.
- Rimossi i record che hanno una lunghezza pari a zero o una media negativa.
- L'errore di ricostruzione è minore usando la normalizzazione minmax, probabilmente perchè i dati non hanno una distribuzione gaussiana
- TIME_STEP maggiori degradano le prestazioni

## Environment setup

In [ ]:
import vitaldb
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import operator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, TimeDistributed, RepeatVector
import tensorflow as tf

from keras import regularizers
from seaborn import histplot



## Data load

### API data load

In [ ]:

def load_from_API(n_cases=None, asa = 3 , op = '<'):

    """ 
    This function loads the VitalDB data from APIs

    INPUT

    - n_cases: limit the number of samples to load
    - asa: ASA status for health status classification, can range in [1,2,3,4,5,6]
    - op: operator to use on asa status. Permits to load a specific portion of the dataset.
    
     
    OUTPUT  

    - dbp: diastolyc blood pressure samples
    - sbp: systolic blood pressure samples
    - bt: body temperature samples
    - hr: heart rate samples
    - rr: respiratory rate samples

    """

    ops = {'>': operator.gt,
       '<': operator.lt,
       '>=': operator.ge,
       '<=': operator.le,
       '==': operator.eq}

    caseids_all = vitaldb.find_cases(['ART_DBP','ART_SBP','BT','HR','RR']) # find ids of patient with this parameters
    
    df = pd.read_csv('https://api.vitaldb.net/cases') # Load dataset
    df = df[ops[op](df['asa'], asa)] # ASA param identifies the health status of the patient

    caseids_unhealthy = df['caseid'].to_numpy() 
    caseids = [el for el in caseids_all if el in caseids_unhealthy]

    if(n_cases is None):
        n_cases = len(caseids)

    dbp = []
    sbp = []
    bt  = []
    hr  = []
    rr  = []

     # load all the patients data 
    for i in range(0,n_cases): # Select only five patient for testing purpose; then len(caseids)
        try:
            vals = vitaldb.load_case(caseids[i], ['ART_DBP','ART_SBP','BT','HR','RR'])
            dbp.append(vals[:,0])
            sbp.append(vals[:,1])
            bt.append(vals[:,2])
            hr.append(vals[:,3])
            rr.append(vals[:,4])
            # extract non-null values
            dbp[i] = dbp[i][~np.isnan(dbp[i])]  
            sbp[i] = sbp[i][~np.isnan(sbp[i])] 
            bt[i] = bt[i][~np.isnan(bt[i])]
            hr[i] = hr[i][~np.isnan(hr[i])] 
            rr[i] = rr[i][~np.isnan(rr[i])]
        except Exception as e: 
            print('\n=================\n')
            print('INDEX: '+str(i))
            print('ERROR: '+str(type(e)))
            print('\n=================\n')
            pass

    
    return dbp,sbp,bt,hr,rr



### Disk data load

In [ ]:
def load_from_disk(path):
    
    dbp = []
    sbp = []
    bt  = []
    hr  = []
    rr  = []

# save the data into a file since loading all the 2k caseids requires at least 1h
    filepath = os.path.join(path,'numeric_data.vitaldb')

    with open(filepath, 'rb') as f:
        (dbp,sbp,bt,hr,rr) = pickle.load(f)
    
    return dbp,sbp,bt,hr,rr

In [ ]:
def save_to_disk(path,dbp,sbp,bt,hr,rr):
# save the data into a file since loading all the 2k caseids requires at least 1h

    with open(os.path.join(path,'numeric_data.vitaldb'), 'wb') as f:
        pickle.dump((dbp,sbp,bt,hr,rr), f)


### Visualize

In [ ]:
def histogram(dbp,sbp,bt,rr,hr):

    """ 
    INPUT 

    - dbp: diastolyc blood pressure samples
    - sbp: systolic blood pressure samples
    - bt: body temperature samples
    - hr: heart rate samples
    - rr: respiratory rate samples

    """


    # remove all the negative values since these vital parameters can only be positive
    flat_list_dbp = np.asarray([item for sublist in dbp for item in sublist if item > 0],dtype='float64')
    flat_list_sbp = np.asarray([item for sublist in sbp for item in sublist if item > 0],dtype='float64')
    flat_list_hr = np.asarray([item for sublist in hr for item in sublist if item > 0],dtype='float64')
    flat_list_bt = np.asarray([item for sublist in bt  for item in sublist if item > 0],dtype='float64')
    flat_list_rr = np.asarray([item for sublist in rr for item in sublist if item > 0],dtype='float64')

    d = {}
    d['Dyastolic BP'] = flat_list_dbp[:]
    d['Systolic BP'] = flat_list_sbp[:]
    d['Heart rate'] = flat_list_hr[:]

    b = {}
    b['Body temperature'] = flat_list_bt[:]
    b['Respiratory rate'] = flat_list_rr[:]

    f,ax = plt.subplots(2,1,figsize=(10,10))
    histplot(d,binwidth=1,ax=ax[0])
    histplot(b,binwidth=1,ax=ax[1])



def plot_data(dbp,sbp,bt,hr,rr,bins=10):
    """ 
    INPUT 

    - dbp: diastolyc blood pressure samples
    - sbp: systolic blood pressure samples
    - bt: body temperature samples
    - hr: heart rate samples
    - rr: respiratory rate samples

    """

    flat_list_dbp = np.asarray([item for sublist in dbp for item in sublist if item > 0 ],dtype='float64')
    flat_list_sbp = np.asarray([item for sublist in sbp for item in sublist if item > 0],dtype='float64')
    flat_list_bt = np.asarray([item for sublist in bt for item in sublist if item > 0],dtype='float64')
    flat_list_hr = np.asarray([item for sublist in hr for item in sublist if item > 0],dtype='float64')
    flat_list_rr = np.asarray([item for sublist in rr for item in sublist if item > 0],dtype='float64')
    

    plt.subplots_adjust(hspace=1.)
    plt.subplot(511)
    plt.title("DBP")
    plt.hist(flat_list_dbp,bins=bins)

    plt.subplots_adjust(hspace=1.)
    plt.subplot(512)
    plt.title("SBP")
    plt.hist(flat_list_sbp,bins=bins)

    plt.subplot(513)
    plt.title("Body temperature")
    plt.hist(flat_list_bt,bins=bins)

    plt.subplot(514)
    plt.title("Heart rate")
    plt.hist(flat_list_hr,bins=bins)

    plt.subplot(515)
    plt.title("Respiratory rate")
    plt.hist(flat_list_rr,bins=bins)
    plt.show()

## Preprocessing

In [ ]:
def clean_data(dbp,sbp,bt,hr,rr):
    """ 
    This function removes time series records which have either zero length (empty record) or negative mean value (affected by registration noise)

    INPUT 

    - dbp: diastolyc blood pressure samples
    - sbp: systolic blood pressure samples
    - bt: body temperature samples
    - hr: heart rate samples
    - rr: respiratory rate samples

    OUTPUT: cleaned data

    NB: Blood pressure (dbp,sbp) ranges accordingly with the European guidelines ESC-ESH 2018.
        Body temperature ranges in 35-37 
        Hearth rate ranges in 60-100
        Respiratory rate ranges in 10-21

        These values for ranges are a bit wider than optimal ones since we want to preserve some variablity in the data. 
     
    """


    try:
        for i in range(0,len(dbp)):
            dbp[i] = dbp[i][dbp[i] > 50]
            dbp[i] = dbp[i][dbp[i] < 90]  # we want to preserve some variability

        for i in range(0,len(dbp) - len([el for el in dbp if len(el) == 0 or (np.mean(el) <= 0)])):
            if(len(dbp[i])==0 or (np.mean(dbp[i]) <= 0)):
                dbp.pop(i)

    except:
        pass

    try:
        for i in range(0,len(sbp)):
            sbp[i] = sbp[i][sbp[i] > 80]
            sbp[i] = sbp[i][sbp[i] < 140] # we want to preserve some variability

        for i in range(0,len(sbp) - len([el for el in sbp if len(el) == 0 or (np.mean(el) <= 0)])):
            if(len(sbp[i])==0 or (np.mean(sbp[i]) <= 0)):
                sbp.pop(i)
    except:
        pass

    try:
        for i in range(0,len(bt)):
            bt[i] = bt[i][bt[i] > 35]
            bt[i] = bt[i][bt[i] < 37.2]
        for i in range(0,len(bt) - len([el for el in bt if len(el) == 0 or (np.mean(el) <= 0)])):
            if(len(bt[i])==0 or (np.mean(bt[i]) <= 0)):
                bt.pop(i)
    except:
        pass

    try:
        for i in range(0,len(hr)):
            hr[i] = hr[i][hr[i] > 60]
            hr[i] = hr[i][hr[i] < 100] # we want to preserve some variability

        for i in range(0,len(hr) - len([el for el in hr if len(el) == 0 or (np.mean(el) <= 0)])):
            if(len(hr[i])==0 or (np.mean(hr[i]) <= 0)):
                hr.pop(i)
    except:
        pass

    try:
        for i in range(0,len(rr)):
            rr[i] = rr[i][rr[i] > 10]
            rr[i] = rr[i][rr[i] < 21] # we want to preserve some variability

        for i in range(0,len(rr) - len([el for el in rr if len(el) == 0 or (np.mean(el) <= 0)])):
            if(len(rr[i])==0 or (np.mean(rr[i]) <= 0)):
                rr.pop(i)
    except:
        pass

    return dbp,sbp,bt,hr,rr


def mean_norm(dbp,sbp,bt,hr,rr):

    # remove all the negative values since these vital parameters can only be positive
    flat_list_dbp = np.asarray([item for sublist in dbp for item in sublist if item > 0],dtype='float64')
    flat_list_sbp = np.asarray([item for sublist in sbp for item in sublist if item > 0],dtype='float64')
    flat_list_hr = np.asarray([item for sublist in hr for item in sublist if item > 0],dtype='float64')
    flat_list_bt = np.asarray([item for sublist in bt  for item in sublist if item > 0],dtype='float64')
    flat_list_rr = np.asarray([item for sublist in rr for item in sublist if item > 0],dtype='float64')
    print({'dbp':len(dbp),'sbp':len(sbp),'hr':len(hr),'bt':len(bt),'rr':len(rr)})



    dbp_mean,dbp_std = np.mean(flat_list_dbp),np.std(flat_list_dbp)
    sbp_mean,sbp_std = np.mean(flat_list_sbp),np.std(flat_list_sbp)
    bt_mean,bt_std = np.mean(flat_list_bt),np.std(flat_list_bt)
    hr_mean,hr_std = np.mean(flat_list_hr),np.std(flat_list_hr)
    rr_mean,rr_std = np.mean(flat_list_rr),np.std(flat_list_rr)

    print('\nMean values for features:')
    print({'dbp':dbp_mean,'sbp':sbp_mean,'hr':hr_mean,'bt':bt_mean,'rr':rr_mean})
    print('\nStd values for features:')
    print({'dbp':dbp_std,'sbp':sbp_std,'hr':hr_std,'bt':bt_std,'rr':rr_std})
    print('\n')

    # Consider runtime warnings such as Divide by zero as Exceptions to throw
    old_settings = np.seterr(divide='raise')
    idx_remove = {}
    idx_remove['sbp'] = []
    idx_remove['dbp'] = []
    idx_remove['bt'] = []
    idx_remove['rr'] = []
    idx_remove['hr'] = []
    

    for i in range(0,len(dbp)): 
        try:
            dbp[i] = (dbp[i] - dbp_mean)/(dbp_std) 
        except: 
            # remove values for which the normalization gives Runtime warning
            idx_remove['dbp'].append(i)



    for i in range(0,len(sbp)): 
        try:
            sbp[i] = (sbp[i] - sbp_mean)/(sbp_std) 
        except: 
            # remove values for which the normalization gives Runtime warning
            idx_remove['sbp'].append(i)
        


    for i in range(0,len(bt)): 
        try:
            bt[i] = (bt[i] - bt_mean)/(bt_std) 
        except: 
            # remove values for which the normalization gives Runtime warning
            idx_remove['bt'].append(i)


    for i in range(0,len(hr)): 
        try:
            hr[i] = (hr[i] - hr_mean)/(hr_std) 
        except: 
            # remove values for which the normalization gives Runtime warning
            idx_remove['hr'].append(i)

    for i in range(0,len(rr)): 
        try:
            rr[i] = (rr[i] - rr_mean)/(rr_std) 
        except: 
            # remove values for which the normalization gives Runtime warning
            idx_remove['hr'].append(i)

    # Back to default settings for errors
    np.seterr(**old_settings)

    return dbp,sbp,bt,hr,rr,idx_remove





def minmax_norm(dbp,sbp,bt,hr,rr):


    # remove all the negative values since these vital parameters can only be positive
    flat_list_dbp = np.asarray([item for sublist in dbp for item in sublist if item > 0],dtype='float64')
    flat_list_sbp = np.asarray([item for sublist in sbp for item in sublist if item > 0],dtype='float64')
    flat_list_hr = np.asarray([item for sublist in hr for item in sublist if item > 0],dtype='float64')
    flat_list_bt = np.asarray([item for sublist in bt  for item in sublist if item > 0] ,dtype='float64')
    flat_list_rr = np.asarray([item for sublist in rr for item in sublist if item > 0],dtype='float64')
    #print({'dbp':len(flat_list_dbp),'sbp':len(flat_list_sbp),'hr':len(flat_list_hr),'bt':len(flat_list_bt),'rr':len(flat_list_rr)})


    dbp_min,dbp_max = np.min(flat_list_dbp),np.max(flat_list_dbp)
    sbp_min,sbp_max = np.min(flat_list_sbp),np.max(flat_list_sbp)
    bt_min,bt_max = np.min(flat_list_bt),np.max(flat_list_bt)
    hr_min,hr_max = np.min(flat_list_hr),np.max(flat_list_hr)
    rr_min,rr_max = np.min(flat_list_rr),np.max(flat_list_rr)
    
    old_settings = np.seterr(divide='raise')
    idx_remove = {}

    idx_remove['sbp'] = []
    idx_remove['dbp'] = []
    idx_remove['bt'] = []
    idx_remove['rr'] = []
    idx_remove['hr'] = []

    for i in range(0,len(dbp)): 
        try:
            dbp[i] = (dbp[i] - dbp_min)/(dbp_max - dbp_min) 
        except: 
            # remove values for which the normalization gives Runtime warning
            idx_remove['dbp'].append(i)


    for i in range(0,len(sbp)): 
        try:
            sbp[i] = (sbp[i] - sbp_min)/(sbp_max - sbp_min) 
        except: 
            # remove values for which the normalization gives Runtime warning
            idx_remove['sbp'].append(i)


    for i in range(0,len(bt)): 
        try:
            bt[i] = (bt[i] - bt_min)/(bt_max - bt_min) 
        except: 
            # remove values for which the normalization gives Runtime warning
            idx_remove['bt'].append(i)


    for i in range(0,len(hr)): 
        try:
            hr[i] = (hr[i] - hr_min)/(hr_max - hr_min) 
        except: 
            # remove values for which the normalization gives Runtime warning
            idx_remove['hr'].append(i)


    for i in range(0,len(rr)): 
        try:
            rr[i] = (rr[i] - rr_min)/(rr_max - rr_min) 
        except: 
            # remove values for which the normalization gives Runtime warning
            idx_remove['rr'].append(i)


    # Back to default settings for errors
    np.seterr(**old_settings)

    return dbp,sbp,bt,hr,rr,idx_remove



def get_preprocessed_data(path, raw = False, norm_func = None):

    dbp,sbp,bt,hr,rr = load_from_disk(path=path)
    
    if(not raw):
        dbp,sbp,bt,hr,rr = clean_data(dbp,sbp,bt,hr,rr)

    idx_remove = []

    if(norm_func is not None):
        dbp,sbp,bt,hr,rr,idx_remove = norm_func(dbp,sbp,bt,hr,rr)
        
    return np.asarray(dbp),np.asarray(sbp),np.asarray(bt),np.asarray(hr),np.asarray(rr),idx_remove

In [ ]:
path = '/Users/Roberto/projects/AnomalyDetection/data/processed'
norm = 'mean'
try:
    func_norm = norm + '_norm'
    normalize = globals()[func_norm] # normalize is called later for test data normalization
except:
    normalize = None

dbp,sbp,bt,hr,rr,idx_remove = get_preprocessed_data(path = path,norm_func = normalize)

# LSTM Autoencoder

## Declarations

### Functions

In [ ]:
# TIME_STEP of preprocess function should be equal to the minimum length of features sequences
min_seq_len = np.min([np.mean([len(el) for el in dbp]),np.mean([len(el) for el in sbp]),np.mean([len(el) for el in bt]),np.mean([len(el) for el in rr]),np.mean([len(el) for el in hr])])
print('Mean length of sequences between features')
min_seq_len

In [ ]:
def preprocess(dbp,sbp,bt,hr,rr, TIME_STEP = 5500, n_instances = None):

    """
    This function prepare the data to be input to the LSTM autoencoder, both for training or testing purpose.
    This function assumes that the input data is already normalized, thus NO values modifications are needed when flattening.

    INPUT 

    - dbp: diastolyc blood pressure samples
    - sbp: systolic blood pressure samples
    - bt: body temperature samples
    - hr: heart rate samples
    - rr: respiratory rate samples

    - TIME_STEP: length of sequences for lstm autoencoder. Default to 5500 since it's the mean length of features sequences
    - n_instances: number of instances to retain. Useful for testing purposes i.e. training the data on a smaller portion of data to computational resources and testing time.

    OUTPUT

    - X: tensor to be input to the LSTM autoencoder

    """

    # flat the data to be later chuncked in sequences

    flat_list_dbp = np.asarray([item for sublist in dbp for item in sublist],dtype='float64')
    flat_list_sbp = np.asarray([item for sublist in sbp for item in sublist],dtype='float64')
    flat_list_bt = np.asarray([item for sublist in bt for item in sublist],dtype='float64')
    flat_list_hr = np.asarray([item for sublist in hr for item in sublist],dtype='float64')
    flat_list_rr = np.asarray([item for sublist in rr for item in sublist],dtype='float64')



    X_dbp = []
    X_sbp = []
    X_bt = []
    X_hr = []
    X_rr = []

    for seq in range(0,len(flat_list_dbp), TIME_STEP):
        X_dbp.append(flat_list_dbp[seq:seq+TIME_STEP])

    for seq in range(0,len(flat_list_sbp), TIME_STEP):
        X_sbp.append(flat_list_sbp[seq:seq+TIME_STEP])

    for seq in range(0,len(flat_list_bt), TIME_STEP):
        X_bt.append(flat_list_bt[seq:seq+TIME_STEP])

    for seq in range(0,len(flat_list_hr), TIME_STEP):
        X_hr.append(flat_list_hr[seq:seq+TIME_STEP])

    for seq in range(0,len(flat_list_rr), TIME_STEP):
        X_rr.append(flat_list_rr[seq:seq+TIME_STEP])


    X_dbp = np.asarray(X_dbp,dtype=object)
    X_sbp = np.asarray(X_sbp,dtype=object)
    X_bt = np.asarray(X_bt,dtype=object)
    X_hr = np.asarray(X_hr,dtype=object)
    X_rr = np.asarray(X_rr,dtype=object)

    # Pad with mean value of data
    X_dbp = pad_sequences(X_dbp, TIME_STEP,padding='post',value = np.mean(flat_list_dbp),dtype='float64')
    X_sbp = pad_sequences(X_sbp, TIME_STEP,padding='post',value = np.mean(flat_list_sbp),dtype='float64')
    X_bt = pad_sequences(X_bt, TIME_STEP,padding='post',value = np.mean(flat_list_bt),dtype='float64')
    X_hr = pad_sequences(X_hr, TIME_STEP,padding='post',value = np.mean(flat_list_hr),dtype='float64')
    X_rr = pad_sequences(X_rr, TIME_STEP,padding='post',value = np.mean(flat_list_rr),dtype='float64')



    X_dbp = np.asarray(np.expand_dims(X_dbp,axis=2))
    X_sbp = np.asarray(np.expand_dims(X_sbp,axis=2))
    X_bt = np.asarray(np.expand_dims(X_bt,axis=2))
    X_hr = np.asarray(np.expand_dims(X_hr,axis=2))
    X_rr = np.asarray(np.expand_dims(X_rr,axis=2))

    # Truncate on the minimum length of features sequences
    min_len = np.min([X_dbp.shape[0],X_sbp.shape[0],X_bt.shape[0],X_hr.shape[0],X_rr.shape[0]])
    
    X_dbp = X_dbp[:min_len,:]
    X_sbp = X_sbp[:min_len,:]
    X_bt = X_bt[:min_len,:]
    X_hr = X_hr[:min_len,:]
    X_rr = X_rr[:min_len,:]

    Y = np.concatenate([X_dbp, X_sbp, X_bt, X_hr, X_rr],axis=2)
    
    if(isinstance(n_instances,int) and n_instances > 0):
        X = Y[:n_instances] # Select the first n instances
    else:
        X = Y

    return X



def eval_error(X_test,res,std=False):

    """
    This function evaluates the reconstruction error of the input samples, 
    returning the mean reconstruction error for every feature and for the overall samples.

    INPUT

    - X_test : test samples
    - res_test: reconstructed test samples
    - std: Boolean, return standard devation for reconstruction error of the features

    OUTPUT: 

    - overall_mean: mean reconstruction error for the entire input data
    - dict_means: Dictionary, contains the mean reconstruction error for the single features


    """

    n = min(X_test.shape[0],res.shape[0])
    n_features = min(X_test.shape[2],res.shape[2])

    overall_mean = 0

    dict_means = {}
    dict_means['sbp'] = 0
    dict_means['dbp'] = 0
    dict_means['bt'] = 0
    dict_means['rr'] = 0
    dict_means['hr'] = 0

    dic_means = {}
    dic_means[0] = 0
    dic_means[1] = 0
    dic_means[2] = 0
    dic_means[3] = 0
    dic_means[4] = 0

    if(std):
        dict_std = {}
        dict_std['sbp'] = 0
        dict_std['dbp'] = 0
        dict_std['bt'] = 0
        dict_std['rr'] = 0
        dict_std['hr'] = 0

        dic_std = {}
        dic_std[0] = 0
        dic_std[1] = 0
        dic_std[2] = 0
        dic_std[3] = 0
        dic_std[4] = 0
    

    for i in range(0,n_features):

        tot = 0
        for p in range(0,n):
            re = np.linalg.norm(X_test[p,:,i] - res[p,:,i],2)
            tot = tot + re            
            dic_means[i] = dic_means[i] + re
 
        mean = np.round(tot/n,3)
        overall_mean = overall_mean + mean
        
        if(std):
            # compute the variance
            for p in range(0,n):
                re = np.linalg.norm(X_test[p,:,i] - res[p,:,i],2)
                dic_std[i] = (re - mean)**2 + dic_std[i]



    dict_means['sbp'] = np.round(dic_means[0]/n,4)
    dict_means['dbp'] = np.round(dic_means[1]/n,4)
    dict_means['bt']  = np.round(dic_means[2]/n,4)
    dict_means['rr']  = np.round(dic_means[3]/n,4)
    dict_means['hr']  = np.round(dic_means[4]/n,4)

    if(std):
        dict_std['sbp'] = np.round((dic_std[0]/n)**(1/2),4)
        dict_std['dbp'] = np.round((dic_std[1]/n)**(1/2),4)
        dict_std['bt']  = np.round((dic_std[2]/n)**(1/2),4)
        dict_std['rr']  = np.round((dic_std[3]/n)**(1/2),4)
        dict_std['hr']  = np.round((dic_std[4]/n)**(1/2),4)
        del dic_std

    del dic_means
    

    overall_mean = overall_mean/n_features

    if(std):
        return overall_mean,dict_means,dict_std
    else:
        return overall_mean,dict_means



def select_threshold(X, res, mean_rec_err, perc=.98, nit=10000, step=None, out=False):

    """
    INPUT

    - X : training data of the autoencoder
    - res: reconstructed data from autoencoder
    - mean_rec_err:  mean reconstruction error obtained on training data
    - perc: percentage of training data for which the reconstruction error must be below the mean_rec_err multiplied by a factor
    - nit: max number of iterations
    - step: step with which increase the multiplication factor at every step

    OUTPUT: 

    - threshold: threshold for reconstruction error on test data, for which considering a sample anomalous

    NB: An higher percentage value will increase the threshold, resulting in a looser anomaly detection. 

    """

    counter = 0
    target = 0
    dim = X.shape[0]
    if(step is None):
        step = 1 - perc # step estimation from perc for a better accuracy in factor estimation
    factor = 1 - step # This subtraction is needed for the factor to be equal to one at the first step of while cycle
    
    while( target < perc and counter < nit):
        j = 0
        counter = counter + 1
        factor = factor + step

        for n in range(dim):
            err = np.linalg.norm(X[n,...]-res[n,...],2)
            if(abs(err <= mean_rec_err * factor)):
                j = j+1
        target = np.round(j/dim,2)
        if(out):
            print(target,factor)
    
    threshold = np.round(factor * mean_rec_err,3)
    return threshold



def select_features_threshold(X, res, dict_means, perc=.98, nit=10000, step=None, out=False):

    """
    INPUT

    - X : training data of the autoencoder
    - res: reconstructed data from autoencoder
    - mean_rec_err:  mean reconstruction error obtained on training data
    - perc: percentage of training data for which the reconstruction error must be below the mean_rec_err multiplied by a factor
    - nit: max number of iterations
    - step: step with which increase the multiplication factor at every step

    OUTPUT: 

    - thresholds: threshold for reconstruction error on test data, for which considering a sample anomalous

    Note: An anomaly will be a sample that differs from the percentage of data specified.

    """


    dim = X.shape[0]
    n_features = X.shape[2]
    means_rec_err = [dict_means[k] for k in dict_means.keys()]
    factors = np.zeros(len(means_rec_err))
    thresholds = np.zeros(len(means_rec_err))

    if(step is None):
        step = 1 - perc # step estimation from perc for a better accuracy in factor estimation
    factors[:] = 1 - step
    
    for f in range(0,n_features):

        counter = 0 # iterations counter
        target = 0
        if(out):
            print('\n\t\t\tFEATURE ' + str(f))
        while( target < perc and counter < nit):

            j = 0 # numerator of percentage estimation j/dim
            counter = counter + 1
            factors[f] = factors[f] + step

            for n in range(dim):
                err = np.linalg.norm(X[n,:,f]-res[n,:,f],2)
                if(abs(err <= means_rec_err[f]*factors[f])):
                    j = j+1

            target = np.round(j/dim,2)
            if(out):
                print(target,factors[f])

        thresholds[f] = np.round(factors[f]*means_rec_err[f],3)

    return thresholds



def detect_anomalies(X_test,res_test,threshold,out=True):
    """
    INPUT

    - X_test : test samples
    - res_test: reconstructed test samples
    - threshold: threshold for reconstruction error on test data, for which considering a sample anomalous. See select_threshold for details

    OUTPUT: 

    - idx_anomalies: list of indices of test samples deemed anomalous on the basis of the given threshold.


    """

    dim = X_test.shape[0]
    idx_anomalies = []

    for i in range(dim):
        err = np.linalg.norm(X_test[i,...]-res_test[i,...],2) 
        if(abs(err > threshold)):
            idx_anomalies.append(i)
    
    if(out):
        print('\nPercentage of anomalies in tested data:' + str(np.round(len(idx_anomalies)/X_test.shape[0],2)))

    return idx_anomalies



def detect_features_anomalies(X_test,res_test,thresholds,out=False):

    """
    INPUT

    - X_test : test samples
    - res_test: reconstructed test samples
    - threshold: threshold for reconstruction error on test data, for which considering a sample anomalous. See select_threshold for details

    OUTPUT: 

    - idx_anomalies: list of indices of test samples deemed anomalous on the basis of the given threshold.


    """

    dim,n_features = X_test.shape[0],X_test.shape[2]
    idx_anomalies = {}
    idx_anomalies['sbp'] = []
    idx_anomalies['dbp'] = []
    idx_anomalies['bt'] = []
    idx_anomalies['rr'] = []
    idx_anomalies['hr'] = []

    dic = {}
    dic[0] = []
    dic[1] = []
    dic[2] = []
    dic[3] = []
    dic[4] = []

    for f in range(0,n_features):
        for i in range(dim):
            err = np.linalg.norm(X_test[i,:,f] - res_test[i,:,f],2) 
            if(abs(err > thresholds[f])):
                dic[f].append(i)

    idx_anomalies['sbp'] = dic[0]
    idx_anomalies['dbp'] = dic[1]
    idx_anomalies['bt'] = dic[2]
    idx_anomalies['rr'] = dic[3]
    idx_anomalies['hr'] = dic[4]

    if(out):
        print('\nPercentage of anomalies in tested data:')
        for i in range(0,n_features):
            print('\nFEATURE ' + str(i) + ': ' + str(np.round(len(dic[i])/X_test.shape[0],2)))

    del dic
    return idx_anomalies


def plot_predict(X_test, res, figsize=(40,40), n = None, single = False, vals = None):

    """
    INPUT

    - X_test : data samples to plot
    - res_test: reconstructed data samples to plot
    - figsize: size of the plot
    - n: number of samples to plot
    - single: Boolean, when True plots the single instance number specified by 'n'
    - vals: List of indices of samples to plot

    """

    plt.figure(figsize=figsize)
    
    if(n is not None and isinstance(n,int) and n > -1):
        if(single):
            for j in range(0,5):
                s = '61'+str(j+1)
                plt.subplot(int(s))
                plt.subplots_adjust(hspace=1.)
                plt.title('Feature ' + str(j+1))
                plt.plot(res[n,:,j],'-r',linewidth=2.5,)
                plt.plot(X_test[n,:,j],'-g',linewidth=2.5,alpha=0.8)
            plt.show()
        else:
            for j in range(0,5):
                for i in range(0,n):
                    s = '61'+str(j+1)
                    plt.subplot(int(s))
                    plt.subplots_adjust(hspace=1.)
                    plt.title('Feature ' + str(j+1))
                    plt.plot(res[i,:,j],'-r',linewidth=2.5,)
                    plt.plot(X_test[i,:,j],'-g',linewidth=2.,alpha=0.5)
            plt.show()
    elif(vals is not None):
        for j in range(0,5):
            for i in vals:
                s = '61'+str(j+1)
                plt.subplot(int(s))
                plt.subplots_adjust(hspace=1.)
                plt.title('Feature ' + str(j+1))
                plt.plot(res[i,:,j],'-r',linewidth=2.5,)
                plt.plot(X_test[i,:,j],'-g',linewidth=2.,alpha=0.5)

        plt.show()
    else:
        n = X_test.shape[0]
        for j in range(0,5):
            for i in range(0,n):
                s = '61'+str(j+1)
                plt.subplot(int(s))
                plt.subplots_adjust(hspace=1.)
                plt.title('Feature ' + str(j+1))
                plt.plot(res[i,:,j],'-r',linewidth=2.5,)
                plt.plot(X_test[i,:,j],'-g',linewidth=2.,alpha=0.5)

        plt.show()

### First model

In [ ]:
X_train = preprocess(dbp,sbp,bt,hr,rr)
batch_size, seq_len, n_features = X_train.shape

model = Sequential()
model.add(LSTM(256, bias_regularizer=regularizers.l2(0.7),  recurrent_regularizer=regularizers.l2(0.6) , input_shape=(seq_len, n_features),return_sequences=True))
model.add(Dropout(rate=0.1))
model.add(LSTM(128,    bias_regularizer=regularizers.l2(0.7),  recurrent_regularizer=regularizers.l2(0.6),   return_sequences=True))
model.add(LSTM(64,  bias_regularizer=regularizers.l1(0.7), recurrent_regularizer=regularizers.l2(0.6),  return_sequences=True))
model.add(tf.keras.layers.LeakyReLU(alpha=0.3))
model.add(LSTM(128,    bias_regularizer=regularizers.l2(0.7),  recurrent_regularizer=regularizers.l2(0.6),   return_sequences=True))
model.add(LSTM(256,   bias_regularizer=regularizers.l2(0.7), recurrent_regularizer=regularizers.l2(0.6),  return_sequences=True))
model.add(Dropout(rate=0.1))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

### Second model

Added RepeatVector layer

In [ ]:
X_train = preprocess(dbp,sbp,bt,hr,rr)
batch_size, seq_len, n_features = X_train.shape

model = Sequential()
model.add(LSTM(256, bias_regularizer=regularizers.l2(0.7),  recurrent_regularizer=regularizers.l2(0.6) , input_shape=(seq_len, n_features),return_sequences=True))
model.add(Dropout(rate=0.1))
model.add(LSTM(128,    bias_regularizer=regularizers.l2(0.7),  recurrent_regularizer=regularizers.l2(0.6),   return_sequences=True))
model.add(LSTM(64,  bias_regularizer=regularizers.l1(0.7), recurrent_regularizer=regularizers.l2(0.6),  return_sequences=False))
model.add(RepeatVector(seq_len))
model.add(LSTM(64,  bias_regularizer=regularizers.l1(0.7), recurrent_regularizer=regularizers.l2(0.6),  return_sequences=True))
model.add(LSTM(128,    bias_regularizer=regularizers.l2(0.7),  recurrent_regularizer=regularizers.l2(0.6),   return_sequences=True))
model.add(LSTM(256,   bias_regularizer=regularizers.l2(0.7), recurrent_regularizer=regularizers.l2(0.6),  return_sequences=True))
model.add(Dropout(rate=0.1))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

## Train, save and load

### First model


In [ ]:
model_first = tf.keras.models.load_model("/Users/Roberto/projects/AnomalyDetection/models/first/autoencoder_LSTM_v2")
#hist = model.fit(X_train,X_train, batch_size = 64, validation_split=0.1, verbose=1)
#model.save("/Users/Roberto/projects/AnomalyDetection/models/first/autoencoder_LSTM_v3.h5")
#model.save("/Users/Roberto/projects/AnomalyDetection/models/first/autoencoder_LSTM_v3")

### Second model

In [ ]:
#model_second = tf.keras.models.load_model("/Users/Roberto/projects/AnomalyDetection/models/second/autoencoder_LSTM_v1")
#model_second.fit(X_train,X_train, batch_size = 64, validation_split=0.1, verbose=1)
#model_second.save("/Users/Roberto/projects/AnomalyDetection/models/second/autoencoder_LSTM_v1.h5")

## Predict

### First model

In [ ]:
# NB: if data il loaded from data/raw or from API need to call clean_data() and normalize()
path = '/Users/Roberto/projects/AnomalyDetection/data/processed/'
with open(os.path.join(path,'numeric_data_ills.vitaldb'), 'rb') as f:
    (dbp_asa_gt_3_,sbp_asa_gt_3_,bt_asa_gt_3_,hr_asa_gt_3_,rr_asa_gt_3_ )  = pickle.load(f)

dbp_asa_gt_3,sbp_asa_gt_3,bt_asa_gt_3,hr_asa_gt_3,rr_asa_gt_3 =  (dbp_asa_gt_3_,sbp_asa_gt_3_,bt_asa_gt_3_,hr_asa_gt_3_,rr_asa_gt_3_ )

dbp_asa_eq_3,sbp_asa_eq_3,bt_asa_eq_3,hr_asa_eq_3,rr_asa_eq_3   = load_from_API(asa= 3, op='==', n_cases=100)
dbp_asa_eq_3,sbp_asa_eq_3,bt_asa_eq_3,hr_asa_eq_3,rr_asa_eq_3   = clean_data(dbp_asa_eq_3,sbp_asa_eq_3,bt_asa_eq_3,hr_asa_eq_3,rr_asa_eq_3 )
dbp_asa_eq_3,sbp_asa_eq_3,bt_asa_eq_3,hr_asa_eq_3,rr_asa_eq_3,_ =  normalize(dbp_asa_eq_3,sbp_asa_eq_3,bt_asa_eq_3,hr_asa_eq_3,rr_asa_eq_3 ) # normalize the test instance with the same approach of train ones


In [ ]:
X_train = X_train
#X_test = heldout part...
X_test_asa_eq_3 = preprocess(dbp_asa_eq_3,sbp_asa_eq_3,bt_asa_eq_3,hr_asa_eq_3,rr_asa_eq_3)
X_test_asa_gt_3 = preprocess(dbp_asa_gt_3,sbp_asa_gt_3,bt_asa_gt_3,hr_asa_gt_3,rr_asa_gt_3 )

res_train = model_first.predict(X_train)
res_asa_eq_3 = model_first.predict(X_test_asa_eq_3)
res_asa_gt_3 = model_first.predict(X_test_asa_gt_3)

In [ ]:
mae_train, dict_mae_train       = eval_error(X_train,res_train)
mae_asa_eq_3, dict_mae_asa_eq_3 = eval_error(X_test_asa_eq_3,res_asa_eq_3)
mae_asa_gt_3, dict_mae_asa_gt_3 = eval_error(X_test_asa_gt_3,res_asa_gt_3)

print("MAE on train set:")
print(dict_mae_train )
print("\nMAE on asa equal to 3:")
print(dict_mae_asa_eq_3 )
print("\nMAE on asa greater than 3:")
print(dict_mae_asa_gt_3 )

In [ ]:
threshold = select_threshold(X_train, res_train, mae_train, perc=.98)
idx_anomalies_asa_eq_3 = detect_anomalies(X_test_asa_eq_3, res_asa_eq_3, threshold)
idx_anomalies_asa_gt_3 = detect_anomalies(X_test_asa_gt_3, res_asa_gt_3, threshold)

In [ ]:
# Let's visualize the presumed anomalies
plot_predict(X_test_asa_eq_3,res_asa_eq_3,n=idx_anomalies_asa_eq_3[0],single=True,figsize = (50,50))

#### Multivariate thresholds

In [ ]:
feat_thresholds = select_features_threshold(X_train, res_train, dict_mae_train, perc=.98)
idx_features_anomalies_asa_eq_3  = detect_features_anomalies(X_test_asa_eq_3, res_asa_eq_3, feat_thresholds, out=True)
idx_features_anomalies_asa_gt_3 = detect_features_anomalies(X_test_asa_gt_3, res_asa_gt_3, feat_thresholds, out=True)